In [1]:
from pyspark.sql import SparkSession

scala_version = '2.12'
spark_version = '3.5.0'
packages = [
    f'org.apache.spark:spark-sql-kafka-0-10_{scala_version}:{spark_version}',
    'org.apache.kafka:kafka-clients:3.5.0'
]

spark = (
    SparkSession
    .builder
    .master("spark://spark-master:7077")
    .config('spark.jars.packages', ",".join(packages))
    .getOrCreate()
)

In [6]:
df = (
    spark
        .readStream
        .format("kafka")
        .option("kafka.bootstrap.servers", "kafka-broker:29092")
        .option("subscribe", "web-events")
        .option("startingOffsets", "earliest")
        .load()
)
df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)


In [7]:
(
    df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")
        .writeStream
        .outputMode("append")
        .foreachBatch(lambda batch_df, batch_id: print(f"batch_id: {batch_id}"))
        .start()
        .awaitTermination(5)    
)

batch_id: 0
batch_id: 1
batch_id: 2
batch_id: 3
batch_id: 4
batch_id: 5
batch_id: 6
batch_id: 7


False

In [8]:
spark.stop()

batch_id: 24
